### 检测移动的目标

目标跟踪是对摄像头视频中的移动目标进行定位的过程。  
实时目标跟踪是许多计算机视觉应用的重要任务，如监控、基于感知的用户界面、增强现实、基于对象的视频压缩以及辅助驾驶等。  

实现视频目标跟踪的方法有很多：
* 当跟踪所有移动目标时，帧之间的差异会变的有用；
* 当跟踪视频中移动的手时，基于皮肤颜色的均值漂移方法是最好的解决方案；
* 当知道跟踪对象的一方面时，模板匹配是不错的技术。

OpenCV中常用的运动物体检测有背景差法、帧差法、光流法，运动物体检测广泛应用于视频安全监控、车辆检测等方面。


### 背景分割器(Background Subtractor) 
* K-Nearest (KNN) 
* Mixture of Gaussian (MOG2)
* Geometric Multigid (GMG)


In [9]:
import cv2

# Step1. 构造VideoCapture对象
cap = cv2.VideoCapture('./images/traffic.flv')

# Step2. 创建一个背景分割器
# createBackgroundSubtractorXXX()函数里，可以指定detectShadows的值
# detectShadows=True，表示检测阴影，反之不检测阴影
# KNN背景分割器
# bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)
# MOG2背景分割器
# bs = cv2.createBackgroundSubtractorMOG2()
# MOG2背景分割器
bs = cv2.bgsegm.createBackgroundSubtractorGMG()

while True :
    ret, frame = cap.read() # 读取视频
    fgmask = bs.apply(frame) # 背景分割
    cv2.imshow('frame', frame) # 显示分割结果
    cv2.imshow('result', fgmask) # 显示分割结果
    if cv2.waitKey(100) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2

def detect_video(video):
    camera = cv2.VideoCapture(video)
    history = 20    # 训练帧数

    bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)  # 背景减除器，设置阴影检测
    bs.setHistory(history)

    frames = 0

    while True:
        res, frame = camera.read()

        if not res:
            break

        fg_mask = bs.apply(frame)   # 获取 foreground mask

        if frames < history:
            frames += 1
            continue

        # 对原始帧进行膨胀去噪
        th = cv2.threshold(fg_mask.copy(), 244, 255, cv2.THRESH_BINARY)[1]
        th = cv2.erode(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=2)
        dilated = cv2.dilate(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 3)), iterations=2)
        # 获取所有检测框
        image, contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for c in contours:
            # 获取矩形框边界坐标
            x, y, w, h = cv2.boundingRect(c)
            # 计算矩形框的面积
            area = cv2.contourArea(c)
            if 500 < area < 3000:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("detection", frame)
        cv2.imshow("back", dilated)
        if cv2.waitKey(100) & 0xff == ord('q'):
            break
    camera.release()

video = './images/people.avi'
detect_video(video)

### 光流
光流（optical flow）是目标、场景或摄像机在连续两帧图像间运动时造成的目标的运动。  
光流在图像中的含义就是动作向量（motion vector）（u,v），分别表示位移在x和y方向上的变化率。它是图像在平移过程中的二维矢量场，是通过二维图像来表示物体点三维运动的速度场，反映了微小时间间隔内由于运动形成的图像变化，以确定图像点上的运动方向和运动速率。  

光流提供了恢复运动的线索。  

光流法的通用假设：  

- [亮度恒定] 图像中目标的像素强度在连续帧之间不会发生变化。
- [时间规律] 相邻帧之间的时间足够短，以至于在考虑运行变化时可以忽略它们之间的差异。该假设用于导出下面的核心方程。
- [空间一致性] 相邻像素具有相似的运动。




In [1]:

"""
calcOpticalFlowPyrLK.py:
由于目标对象或者摄像机的移动造成的图像对象在 续两帧图像中的移动 被称为光流。
它是一个 2D 向量场 可以用来显示一个点从第一帧图像到第二 帧图像之间的移动。
光 流在很多领域中都很有用
• 由运动重建结构
• 视频压缩
• Video Stabilization 等
"""
 
'''
重点函数解读：
1. cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)  
用于获得光流估计所需要的角点
参数说明：
old_gray表示输入图片，
mask表示掩模，
feature_params:maxCorners=100角点的最大个数,
qualityLevel=0.3角点品质,minDistance=7即在这个范围内只存在一个品质最好的角点
2. pl, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)  
用于获得光流检测后的角点位置
参数说明：pl表示光流检测后的角点位置，st表示是否是运动的角点，
err表示是否出错，old_gray表示输入前一帧图片，frame_gray表示后一帧图片，
p0表示需要检测的角点，lk_params：winSize表示选择多少个点进行u和v的求解，maxLevel表示空间金字塔的层数
3. cv2.add(frame, mask) # 将两个图像的像素进行加和操作
参数说明：frame表示输入图片，mask表示掩模
光流估计：通过当前时刻与前一时刻的亮度不变的特性
I(x, y, t) = I(x+?x, y+?y, t+?t) 使用lucas-kanade算法进行求解问题， 我们需要求得的是x,y方向的速度
'''
 
import numpy as np
import cv2
 
# 第一步：视频的读入
cap = cv2.VideoCapture('./images/traffic.flv')
 
# 第二步：构建角点检测所需参数
# params for ShiTomasi corner detection
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)
# Parameters for lucas kanade optical flow
# maxLevel 为使用的图像金字塔层数
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
 
# 第三步：拿到第一帧图像并灰度化作为前一帧图片
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# 第四步:返回所有检测特征点，需要输入图片，角点的最大数量，品质因子，minDistance=7如果这个角点里有比这个强的就不要这个弱的
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
# 第五步:创建一个mask, 用于进行横线的绘制
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
 
while True:
    # 第六步：读取图片灰度化作为后一张图片的输入
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # 第七步：进行光流检测需要输入前一帧和当前图像及前一帧检测到的角点
    # calculate optical flow能够获取点的新位置
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # 第八步：读取运动了的角点st == 1表示检测到的运动物体，即v和u表示为0
    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]
    # 第九步：绘制轨迹
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    # 第十步：将两个图片进行结合，并进行图片展示
    img = cv2.add(frame, mask)
    cv2.imshow('frame', img)
 
    k = cv2.waitKey(30) #& 0xff
    if k == 27:
        break
    # 第十一步：更新前一帧图片和角点的位置
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
 
cv2.destroyAllWindows()
cap.release()
 
 
'''
代码：
第一步：使用cv2.capture读入视频
第二步：构造角点检测所需参数, 构造lucas kanade参数
第三步：拿到第一帧图像，并做灰度化， 作为光流检测的前一帧图像
第四步：使用cv2.goodFeaturesToTrack获得光流检测所需要的角点
第五步: 构造一个mask用于画直线
第六步：读取一张图片，进行灰度化，作为光流检测的后一帧图像
第七步：使用cv2.caclOpticalFlowPyrLK进行光流检测
第八步：使用st==1获得运动后的角点，原始的角点位置
第九步：循环获得角点的位置，在mask图上画line，在后一帧图像上画角点
第十步：使用cv2.add()将mask和frame的像素点相加并进行展示
第十一步：使用后一帧的图像更新前一帧的图像，同时使用运动的后一帧的角点位置来代替光流检测需要的角点
'''


error: OpenCV(3.4.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


### 均值飘逸 Meanshift

寻找概率密度函数离散样本的最大密度，并重新计算下一帧中的最大密度，该算法给出了目标移动的方向。


![meanshift](./figures/meanshift.png)
移动蓝色的窗口，使得蓝色圆心与圆内所有点构成的质心重合。在新移动后的圆环区域当中再次寻找圆环当中所包围点集的质心，然后再次移动，通常情况下，型心和质心是不重合的。不断执行上面的移动过程，直到型心和质心大致重合结束。  




In [3]:
import cv2
import numpy as np

# 设置初始化的窗口位置
r,h,c,w = 0,100,0,100 # 设置初试窗口位置和大小
track_window = (c,r,w,h)

cap = cv2.VideoCapture(0)

ret, frame= cap.read()

# 设置追踪的区域
roi = frame[r:r+h, c:c+w]
# roi区域的hsv图像
hsv_roi = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
# 取值hsv值在(0,60,32)到(180,255,255)之间的部分
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
# 计算直方图,参数为 图片(可多)，通道数，蒙板区域，直方图长度，范围
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
# 归一化
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# 设置终止条件，迭代10次或者至少移动1次
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret, frame = cap.read()
    if ret == True:
        # 计算每一帧的hsv图像
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # 计算反向投影
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # 调用meanShift算法在dst中寻找目标窗口，找到后返回目标窗口
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv2.imshow('img2',img2)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 


### 反向投影 Back Projection
反向投影是一种记录给定图像中的像素点如何适应直方图模型像素分布的方式。简单的讲，就是首先计算某一特征的直方图模型，然后使用模型去寻找图像中存在的该特征。  

反向投影其实是直方图运算的逆过程。直方图运算是统计每个灰度值对应的像素个数，而反向投影则是将像素个数回送到该像素个数对应灰度区间的像素位置。

In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def nothing(x):
    pass

img = cv2.imread('./images/hand.jpg')
ROI = cv2.imread('./images/handROI.jpg')

imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
roiHSV = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)
cv2.namedWindow('BackProjection Image')
cv2.createTrackbar('H Value', 'BackProjection Image', 0, 180, nothing)

while(1):

    hSize = cv2.getTrackbarPos('H Value', 'BackProjection Image')
    hSize = max(2, hSize)
    roiHSVHist = cv2.calcHist([roiHSV], [0, 1], None, [hSize, 256], [0, 180, 0, 256])
    cv2.normalize(roiHSVHist, roiHSVHist, 0, 255, cv2.NORM_MINMAX)
    backProjImg = cv2.calcBackProject([imgHSV], [0, 1], roiHSVHist, [0, 180, 0, 256], 1)
    cv2.imshow('BackProjection Image', backProjImg)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

cv2.waitKey()

-1

### CamShift
连续的自适应MeanShift算法，是对MeanShift算法的改进算法，可以在跟踪的过程中随着目标大小的变化实时调整搜索窗口大小，对于视频序列中的每一帧还是采用MeanShift来寻找最优迭代结果

基本思想是以视频图像中运动物体的颜色信息作为特征，对输入图像的每一帧分别作 Mean-Shift 运算，并将上一帧的目标中心和搜索窗口大小(核函数带宽)作为下一帧 Mean shift 算法的中心和搜索窗口大小的初始值，如此迭代下去，就可以实现对目标的跟踪。
 
实现步骤  
第一步：选中物体，记录你输入的方框和物体。  
第二步：求出视频中有关物体的反向投影图。  
第三步：根据反向投影图和输入的方框进行meanshift迭代，由于它是向重心移动，即向反向投影图中概率大的地方移动，所以始终会移动到目标上。  
第四步：然后下一帧图像时用上一帧输出的方框来迭代即可。  


In [1]:
import cv2
import numpy as np

xs, ys, ws, hs = 0, 0, 0, 0  # selection.x selection.y
xo, yo = 0, 0  # origin.x origin.y
selectObject = False
trackObject = 0


def onMouse(event, x, y, flags, prams):
    global xs, ys, ws, hs, selectObject, xo, yo, trackObject
    if selectObject == True:
        xs = min(x, xo)
        ys = min(y, yo)
        ws = abs(x - xo)
        hs = abs(y - yo)
    if event == cv2.EVENT_LBUTTONDOWN:
        xo, yo = x, y
        xs, ys, ws, hs = x, y, 0, 0
        selectObject = True
    elif event == cv2.EVENT_LBUTTONUP:
        selectObject = False
        trackObject = -1


cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cv2.namedWindow('imshow')
cv2.setMouseCallback('imshow', onMouse)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
while (True):
    ret, frame = cap.read()
    if trackObject != 0:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, np.array((0., 30., 10.)), np.array((180., 256., 255.)))
        if trackObject == -1:
            track_window = (xs, ys, ws, hs)
            maskroi = mask[ys:ys + hs, xs:xs + ws]
            hsv_roi = hsv[ys:ys + hs, xs:xs + ws]
            roi_hist = cv2.calcHist([hsv_roi], [0], maskroi, [180], [0, 180])
            cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
            trackObject = 1
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        dst &= mask
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, 255, 2)

    if selectObject == True and ws > 0 and hs > 0:
        cv2.imshow('imshow1', frame[ys:ys + hs, xs:xs + ws])
        cv2.bitwise_not(frame[ys:ys + hs, xs:xs + ws], frame[ys:ys + hs, xs:xs + ws])
    cv2.imshow('imshow', frame)
    if cv2.waitKey(10) == ord('q'):
        break
cv2.destroyAllWindows()


error: OpenCV(3.4.1) C:\projects\opencv-python\opencv\modules\video\src\camshift.cpp:64: error: (-5) Input window has non-positive sizes in function cv::meanShift


### 参考资料
* 光流法运动目标检测 - Gavinmiaoc的博客 - CSDN博客 
https://blog.csdn.net/Gavinmiaoc/article/details/90400668